# Evolver Loop 2 Analysis

## Key Questions:
1. What is the gap to target and where does it come from?
2. Which N values have the most room for improvement?
3. What approaches haven't been tried yet?

In [1]:
import pandas as pd
import numpy as np
from numba import njit
import math

# Tree geometry
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s','')) for v in a], np.float64)

tx, ty = make_polygon_template()
print('Functions defined')

Functions defined


In [2]:
# Load baseline and calculate per-N scores
df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

per_n_scores = []
for n, g in df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc = score_group(xs, ys, ds, tx, ty)
    per_n_scores.append({'N': n, 'score': sc})

per_n_df = pd.DataFrame(per_n_scores)
print(f'Total score: {per_n_df["score"].sum():.6f}')
print(f'Target: 68.919154')
print(f'Gap: {per_n_df["score"].sum() - 68.919154:.6f}')

Total score: 70.676102
Target: 68.919154
Gap: 1.756948


In [3]:
# Analyze score distribution
per_n_df['cumsum'] = per_n_df['score'].cumsum()
per_n_df['pct_of_total'] = per_n_df['score'] / per_n_df['score'].sum() * 100

print('Top 20 score contributors:')
print(per_n_df.nlargest(20, 'score')[['N', 'score', 'pct_of_total']].to_string())

Top 20 score contributors:
     N     score  pct_of_total
0    1  0.661250      0.935606
1    2  0.450779      0.637810
2    3  0.434745      0.615123
4    5  0.416850      0.589803
3    4  0.416545      0.589372
6    7  0.399897      0.565816
5    6  0.399610      0.565411
8    9  0.387415      0.548156
7    8  0.385407      0.545315
14  15  0.379203      0.536536
9   10  0.376630      0.532896
20  21  0.376451      0.532642
19  20  0.376057      0.532085
10  11  0.375736      0.531631
21  22  0.375258      0.530955
15  16  0.374128      0.529356
25  26  0.373997      0.529170
11  12  0.372724      0.527369
12  13  0.372323      0.526802
24  25  0.372144      0.526549


In [4]:
# Calculate efficiency (trees per unit area)
per_n_df['efficiency'] = per_n_df['N'] / (per_n_df['score'] * per_n_df['N'])  # trees / score contribution
per_n_df['side'] = np.sqrt(per_n_df['score'] * per_n_df['N'])  # bounding box side

print('\nLowest efficiency (most room for improvement):')
print(per_n_df.nsmallest(20, 'efficiency')[['N', 'score', 'side', 'efficiency']].to_string())


Lowest efficiency (most room for improvement):
     N     score      side  efficiency
0    1  0.661250  0.813173    1.512287
1    2  0.450779  0.949504    2.218381
2    3  0.434745  1.142031    2.300198
4    5  0.416850  1.443692    2.398947
3    4  0.416545  1.290806    2.400702
6    7  0.399897  1.673104    2.500647
5    6  0.399610  1.548438    2.502438
8    9  0.387415  1.867280    2.581211
7    8  0.385407  1.755921    2.594658
14  15  0.379203  2.384962    2.637111
9   10  0.376630  1.940696    2.655125
20  21  0.376451  2.811667    2.656389
19  20  0.376057  2.742469    2.659172
10  11  0.375736  2.033002    2.661443
21  22  0.375258  2.873270    2.664832
15  16  0.374128  2.446640    2.672882
25  26  0.373997  3.118320    2.673819
11  12  0.372724  2.114873    2.682950
12  13  0.372323  2.200046    2.685839
24  25  0.372144  3.050182    2.687129


In [5]:
# What improvement is needed per N to reach target?
target = 68.919154
current = per_n_df['score'].sum()
gap = current - target

print(f'\nCurrent: {current:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f} ({gap/current*100:.2f}%)')

# If we could improve each N by the same percentage, what would it be?
required_pct_improvement = gap / current * 100
print(f'\nRequired average improvement per N: {required_pct_improvement:.2f}%')


Current: 70.676102
Target: 68.919154
Gap: 1.756948 (2.49%)

Required average improvement per N: 2.49%


In [6]:
# Analyze which N values are likely to have room for improvement
# Large N values with lattice structure might be improvable

large_n = per_n_df[per_n_df['N'] >= 50].copy()
print(f'\nLarge N (>=50) contribution: {large_n["score"].sum():.6f} ({large_n["score"].sum()/current*100:.1f}%)')

small_n = per_n_df[per_n_df['N'] < 50].copy()
print(f'Small N (<50) contribution: {small_n["score"].sum():.6f} ({small_n["score"].sum()/current*100:.1f}%)')


Large N (>=50) contribution: 51.994683 (73.6%)
Small N (<50) contribution: 18.681420 (26.4%)


In [7]:
# Key insight: The gap is 1.76 points (2.5%)
# If we can improve large N values by 5%, we'd gain:
large_n_improvement = large_n['score'].sum() * 0.05
print(f'\n5% improvement on large N (>=50): {large_n_improvement:.6f}')
print(f'This would close {large_n_improvement/gap*100:.1f}% of the gap')

# If we can improve small N values by 5%:
small_n_improvement = small_n['score'].sum() * 0.05
print(f'\n5% improvement on small N (<50): {small_n_improvement:.6f}')
print(f'This would close {small_n_improvement/gap*100:.1f}% of the gap')


5% improvement on large N (>=50): 2.599734
This would close 148.0% of the gap

5% improvement on small N (<50): 0.934071
This would close 53.2% of the gap


In [8]:
# Summary of findings
print('='*60)
print('ANALYSIS SUMMARY')
print('='*60)
print(f'Current score: {current:.6f}')
print(f'Target score: {target:.6f}')
print(f'Gap to close: {gap:.6f} ({gap/current*100:.2f}%)')
print()
print('Key insights:')
print('1. Small N (<50) contributes more to total score')
print('2. Large N (>=50) might have more room for improvement via lattice')
print('3. The pre-optimized solution is at a very tight local optimum')
print('4. Standard SA optimization found NO improvements')
print()
print('Recommended approaches:')
print('1. Implement proper lattice-based SA for large N (egortrushin approach)')
print('2. Try fractional translation with micro-steps')
print('3. Run much longer optimization (hours, not minutes)')
print('4. Use population-based optimization with perturbation')

ANALYSIS SUMMARY
Current score: 70.676102
Target score: 68.919154
Gap to close: 1.756948 (2.49%)

Key insights:
1. Small N (<50) contributes more to total score
2. Large N (>=50) might have more room for improvement via lattice
3. The pre-optimized solution is at a very tight local optimum
4. Standard SA optimization found NO improvements

Recommended approaches:
1. Implement proper lattice-based SA for large N (egortrushin approach)
2. Try fractional translation with micro-steps
3. Run much longer optimization (hours, not minutes)
4. Use population-based optimization with perturbation
